## DATA PROCESSING TASK

##### This notebook presents a data engineering solution to profile, cleanse, deduplicate, and validate UK company data using the Companies House REST API. Starting from a raw CSV file (Company.csv), we standardize and enrich company records, ensuring data quality and accuracy. The final output includes a cleaned dataset, enrichment fields, and visual insights into data quality and API match success. Following steps are incuded: 
##### 1 - Import Libraries and Load Dataset
##### 2 - Profiling
##### 3 - Cleansing
##### 4 - Deduplication
##### 5 - API Data Fetch
##### 6 - Data Merge
##### 7 - Data Matching and Validation
##### 8 - Enrichment
##### 9 - Reporting and Visualization

### 1 - Import Libraries and Load Dataset

In [ ]:
# 1. Import Libraries
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sweetviz as sv
import regex

# 2. Load Dataset
df = pd.read_csv("Company.csv")
# 3. Basic Structure
print("Dataset Shape:", df.shape)
df.info()

### 2 - Data Profiling

In [ ]:
# 4. Preview Data
report = sv.analyze(df)
report.show_html()

In [ ]:
# 5. Summary Statistics (Including Strings)
df.describe(include='all')

In [ ]:
# 6. Missing Values
missing = df.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
print("Columns with Missing Values:\n", missing)

In [ ]:
# 9. Top Company Names (possible duplicates)
print("Top Repeated Company Numbers:\n", df[" CompanyNumber"].value_counts().head(20))

In [ ]:
# 9. Top Company Number (possible duplicates)
print("Top Repeated Company Names:\n", df["CompanyName"].value_counts().head(10))

In [ ]:
# 10. Unique Value Counts Per Column 
unique_counts = df.nunique().sort_values(ascending=False)
print("Unique Values per Column:\n", unique_counts)

### 3 - Data Cleansing 

In [ ]:
# 1. Drop columns where more than 90% values are NaN
threshold = 276  # 90%
null_count = df.isnull().sum()
cols_to_drop = null_count[null_count > threshold].index
print(f"\nDropping columns with more than {90}% missing values: {list(cols_to_drop)}\n")
df.drop(columns=cols_to_drop, inplace=True)

In [ ]:
# 2. Strip whitespace from column names 
df.columns = df.columns.str.strip()

# 3. Trim whitespace in all string/object columns
str_cols = df.select_dtypes(include=['object']).columns
df[str_cols] = df[str_cols].apply(lambda x: x.str.strip())

In [ ]:
# Integer columns
cols_to_int = [
    'Accounts.AccountRefDay',
    'Accounts.AccountRefMonth',
    'Mortgages.NumMortCharges',
    'Mortgages.NumMortOutstanding',
    'Mortgages.NumMortPartSatisfied',
    'Mortgages.NumMortSatisfied',
    'LimitedPartnerships.NumGenPartners',
    'LimitedPartnerships.NumLimPartners'
]
for col in cols_to_int:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')

# Date columns
# 5. Convert common date fields to datetime (ignore errors)
date_columns = [col for col in df.columns if 'Date' in col]
for col in date_columns:
    df[col] = pd.to_datetime(df[col], format='%d/%m/%Y', errors='coerce')

## str columns
str_cols = ['CompanyName', 'CompanyNumber', 'RegAddress.AddressLine1',
       'RegAddress.AddressLine2', 'RegAddress.PostTown',
       'RegAddress.County', 'RegAddress.Country', 'RegAddress.PostCode',
       'CompanyCategory', 'CompanyStatus', 'CountryOfOrigin',
        'Accounts.AccountCategory','SICCode.SicText_1', 'SICCode.SicText_2']
for col in str_cols:
    df[col] = df[col].str.upper().str.strip().str.replace(r'\s+', ' ', regex=True)

In [ ]:
        
df = df.rename(columns={
    'Returns.NextDueDate': 'ReturnsNextDueDate',
    'Returns.LastMadeUpDate': 'ReturnsLastMadeUpDate'
})

#### a - Cleaning key columns

##### - RegAddress.Country

In [ ]:
df['RegAddress.Country'].value_counts()

In [ ]:
# Define mapping rules
country_cleaning_map = {
    'ENGLAND': 'UNITED KINGDOM',
    'UK': 'UNITED KINGDOM',
    'SCOTLAND': 'UNITED KINGDOM',
    'WALES': 'UNITED KINGDOM',
    'NORTHERN IRELAND': 'UNITED KINGDOM',
    '05/02/1980': '',  # Invalid entry
}

# Apply the mapping
df['RegAddress.Country'] = df['RegAddress.Country'].replace(country_cleaning_map)

df['RegAddress.Country'].value_counts()

##### - CompanyCategory

In [ ]:
df['CompanyCategory'].value_counts()

In [ ]:

# Standardization mapping
category_map = {
    'PRIVATE LIMITED COMPANY': 'Private Limited Company',
    'PRIVATE LTD COMPANY': 'Private Limited Company',
    'PRI/LTD BY GUAR/NSC (PRIVATE, LIMITED BY GUARANTEE, NO SHARE CAPITAL)': 'Private Ltd by Guarantee (No Share Capital)',
    'PRI/LTD BY GUAR/NSC (PRIVATE, LTD BY GUARANTEE, NO SHARE CAPITAL)': 'Private Ltd by Guarantee (No Share Capital)',
    'PRI/LBG/NSC (PRIVATE, LIMITED BY GUARANTEE, NO SHARE CAPITAL, USE OF \'LIMITED\' EXEMPTION)': 'Ltd by Guarantee (Limited Exemption)',
    'LIMITED LIABILITY PARTNERSHIP': 'Limited Liability Partnership',
    'LIMITED PARTNERSHIP': 'Limited Partnership',
    'OVERSEAS ENTITY': 'Overseas Entity',
    'OTHER COMPANY TYPE': 'Other Company Type',
    '5': '',  # Invalid entry
}

# Apply the mapping
df['CompanyCategory'] = df['CompanyCategory'].replace(category_map)

# Optional: Capitalize uniformly
df['CompanyCategory'] = df['CompanyCategory'].str.title()

df['CompanyCategory'].value_counts()

##### - CompanyStatus

In [ ]:
df['CompanyStatus'].value_counts()

In [ ]:
# Cleaning map
status_map = {
    'ACTIVE': 'Active',
    'ACTIVE - PROPOSAL TO STRIKE OFF': 'Active (To Strike Off)',
    'ACTIVE, UK"': 'Active',
    'LIQUIDATION': 'Liquidation',
    'VOLUNTARY ARRANGEMENT': 'Voluntary Arrangement',
    '31/03/1988': '',  # Invalid entry
}

# Apply map
df['CompanyStatus'] = df['CompanyStatus'].replace(status_map)

# Optional: Title case everything
df['CompanyStatus'] = df['CompanyStatus'].str.title()
df['CompanyStatus'].value_counts()

##### - CountryOfOrigin

In [ ]:
df['CountryOfOrigin'].value_counts()

In [ ]:
# Define cleaning map
origin_map = {
    'UK': 'United Kingdom',
    'UNITED KINGDOM': 'United Kingdom',
    '31/05/1986': '',  # Invalid entry
}

# Apply map
df['CountryOfOrigin'] = df['CountryOfOrigin'].replace(origin_map)

# Optional: Title case for consistency
df['CountryOfOrigin'] = df['CountryOfOrigin'].str.title()

In [ ]:
df['CountryOfOrigin'].value_counts()

 ##### - Postal Code

In [ ]:
uk_postcode_regex = r"^[A-Z]{1,2}\d{1,2}[A-Z]?\s?\d[A-Z]{2}$"
df['ValidPostcode'] = df['RegAddress.PostCode'].str.upper().str.match(uk_postcode_regex)
df['ValidPostcode'].value_counts()

In [ ]:
cols_to_int = [
    'Accounts.AccountRefDay',
    'Accounts.AccountRefMonth',
    'Mortgages.NumMortCharges',
    'Mortgages.NumMortOutstanding',
    'Mortgages.NumMortPartSatisfied',
    'Mortgages.NumMortSatisfied',
    'LimitedPartnerships.NumGenPartners',
    'LimitedPartnerships.NumLimPartners'
]


##### - Accounts.AccountRefDay

In [ ]:
df['Accounts.AccountRefDay'].fillna(999).value_counts()

In [ ]:
df['Accounts.AccountRefDay'] = df['Accounts.AccountRefDay'].fillna(999)
df['Accounts.AccountRefDay'] = df['Accounts.AccountRefDay'].apply(lambda x: 999 if x > 31 else x)
df['Accounts.AccountRefDay'].value_counts()

##### - Accounts.AccountRefMonth

In [ ]:
df['Accounts.AccountRefMonth'].fillna(999).value_counts()

In [ ]:
df['Accounts.AccountRefMonth'] = df['Accounts.AccountRefMonth'].fillna(999)
df['Accounts.AccountRefMonth'] = df['Accounts.AccountRefMonth'].apply(lambda x: x if 1 <= x <= 12 else 999)
df['Accounts.AccountRefMonth'].value_counts()

In [ ]:
# 6. Show result
print("Cleansing completed. New shape:", df.shape)
# df.head()

In [ ]:
df_sweet = df.select_dtypes(exclude=['boolean', 'Int64'])
report = sv.analyze(df_sweet)
report.show_html()

### 4 - Deduplication

In [ ]:
# Find duplicated CompanyNumber entries (including all duplicates, not just the second+)
duplicates = df[df.duplicated(subset='CompanyNumber', keep=False)]

# Sort for readability
duplicates = duplicates.sort_values(by='CompanyNumber')

# Display duplicated rows
duplicates

In [ ]:
# Step 2: Group by CompanyNumber and merge fields
def merge_group(group):
    # Take the first non-null value from each column
    return group.ffill().bfill().iloc[0]

# Only apply if CompanyNumber column exists
if 'CompanyNumber' in df.columns:
    # Create merged DataFrame by group
    df_unique = df.groupby('CompanyNumber', as_index=False).apply(merge_group)

    # Reset index after groupby
    df_unique.reset_index(drop=True, inplace=True)

    print(f"Deduplication done. Reduced from {df.shape[0]} to {df_unique.shape[0]} rows.")
else:
    print("CompanyNumber column not found for deduplication.")

In [ ]:
df_unique = df_unique[df_unique['CompanyNumber'].notnull()]
len(df_unique)

### 4 - Fetch data via Rest API and pre process it

In [ ]:
import base64
from requests import get
# Your actual API key
api_key = os.environ.get("MY_API_KEY")

# Encode the API key using base64 for Basic Auth
auth_string = f"{api_key}:"
auth_bytes = auth_string.encode("utf-8")
auth_base64 = base64.b64encode(auth_bytes).decode("utf-8")

# Set Authorization header manually
headers = {
    "Authorization": f"Basic {auth_base64}",
    "Accept": "application/json"
}

In [ ]:
import requests
import time

# Assume df contains a column 'CompanyNumber'
company_numbers = df_unique['CompanyNumber'].dropna().unique()

api_results = []

# Loop through each CompanyNumber
for number in company_numbers:
    try:
        url = f"http://data.companieshouse.gov.uk/doc/company/{number}"
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            json_data = response.json()

            if 'primaryTopic' in json_data:
                api_results.append(json_data)
                print(f"Fetched data for CompanyNumber: {number}")
            else:
                print(f"No 'primaryTopic' found in response for {number}")

        else:
            print(f"Failed to fetch {number} — Status Code: {response.status_code}")

        time.sleep(0.2)  # be nice to the server

    except Exception as e:
        print(f"Error fetching {number}: {e}")


In [ ]:
# Flatten into list of rows
rows = []
for item in api_results:
    p = item['primaryTopic']
    row = {
        'CompanyName': p.get('CompanyName'),
        'CompanyNumber': p.get('CompanyNumber'),
        'AddressLine1': p.get('RegAddress', {}).get('AddressLine1'),
        'PostTown': p.get('RegAddress', {}).get('PostTown'),
        'Country': p.get('RegAddress', {}).get('Country'),
        'Postcode': p.get('RegAddress', {}).get('Postcode'),
        'CompanyCategory': p.get('CompanyCategory'),
        'CompanyStatus': p.get('CompanyStatus'),
        'CountryOfOrigin': p.get('CountryOfOrigin'),
        'IncorporationDate': p.get('IncorporationDate'),
        'AccountRefDay': p.get('Accounts', {}).get('AccountRefDay'),
        'AccountRefMonth': p.get('Accounts', {}).get('AccountRefMonth'),
        'NextAccountsDueDate': p.get('Accounts', {}).get('NextDueDate'),
        'LastAccountsMadeUpDate': p.get('Accounts', {}).get('LastMadeUpDate'),
        'ReturnsNextDueDate': p.get('Returns', {}).get('NextDueDate'),
        'ReturnsLastMadeUpDate': p.get('Returns', {}).get('LastMadeUpDate'),
        'SICCodes': ", ".join(p.get('SICCodes', {}).get('SicText', []))
    }
    rows.append(row)

# Create DataFrame
df_api = pd.DataFrame(rows)

# 2. Strip whitespace from column names (optional but helpful)
df_api.columns = df_api.columns.str.strip()

# 3. Trim whitespace in all string/object columns
str_cols = df_api.select_dtypes(include=['object']).columns
df_api[str_cols] = df_api[str_cols].apply(lambda x: x.str.strip())

# List of columns to convert 
date_columns = [
    'IncorporationDate',
    'NextAccountsDueDate',
    'LastAccountsMadeUpDate',
    'ReturnsNextDueDate',
    'ReturnsLastMadeUpDate'
]

# Apply conversion safely with error handling
for col in date_columns:
    if col in df_api.columns:
        df_api[col] = pd.to_datetime(df_api[col], format='%d/%m/%Y', errors='coerce')

cols_to_int = [
    'AccountRefDay',
    'AccountRefMonth',
   ]
for col in cols_to_int:
    if col in df_api.columns:
        df_api[col] = pd.to_numeric(df_api[col], errors='coerce').astype('Int64')


suffix = '_api'

df_api = df_api.rename(columns={
    col: col + suffix for col in df_api.columns if col != 'CompanyNumber'
})
# Show
# df_api.head()


In [ ]:
df_api.shape

### 5 - Merge both dfs to perform Matching and Validation

In [ ]:
df_original=df_unique
# Ensure CompanyNumber is string and used as merge key
df_original['CompanyNumber'] = df_original['CompanyNumber'].astype(str).str.strip()
df_api['CompanyNumber'] = df_api['CompanyNumber'].astype(str).str.strip()

# Merge original and API data on CompanyNumber
merged_df = pd.merge(
    df_original, df_api,
    on='CompanyNumber',
    how='left'
)

### 6 - Matching

In [ ]:
# Define your mappings
column_map = {
    'CompanyName': 'CompanyName_api',
    'RegAddress.AddressLine1': 'AddressLine1_api',
    'RegAddress.PostTown': 'PostTown_api',
    'RegAddress.Country': 'Country_api',
    'RegAddress.PostCode': 'Postcode_api',
    'CompanyCategory': 'CompanyCategory_api',
    'CompanyStatus': 'CompanyStatus_api',
    'CountryOfOrigin': 'CountryOfOrigin_api',
    'IncorporationDate': 'IncorporationDate_api',
    'Accounts.AccountRefDay': 'AccountRefDay_api',
    'Accounts.AccountRefMonth': 'AccountRefMonth_api',
    'Accounts.NextDueDate': 'NextAccountsDueDate_api',
    'Accounts.LastMadeUpDate': 'LastAccountsMadeUpDate_api',
    'ReturnsNextDueDate': 'ReturnsNextDueDate_api',
    'ReturnsLastMadeUpDate': 'ReturnsLastMadeUpDate_api'
}


In [ ]:
validated_df = merged_df.copy()

for orig_col, api_col in column_map.items():
    qa_col = orig_col.split('.')[-1] + '_QA'  # cleaner QA column name

    def compare_values(row, orig=orig_col, api=api_col):
        val1 = row.get(orig)
        val2 = row.get(api)
        try:
            # 1. Return "Missing in both sources"
            if pd.isna(val1) and pd.isna(val2):
                return 'Missing in both sources'

            # 2. Missing in original only
            if pd.isna(val1) and pd.notna(val2):
                return 'Missing in original source'

            # 3. Missing in API only
            if pd.isna(val2) and pd.notna(val1):
                return 'Missing in api source'

            # 4. Exact match
            if val1 == val2:
                return "Matched"

            # 5. Mismatch
            return "Not Matched"
        except Exception as e:
            return "Error"

    validated_df[qa_col] = validated_df.apply(compare_values, axis=1)


In [ ]:
# Show QA columns
qa_columns = [col for col in validated_df.columns if col.endswith('_QA')]
validated_df[qa_columns].sample(6)


### 7 - Enrichment

##### - Update Missing and Mismatching values in original df with values from api results

In [ ]:
for orig_col, api_col in column_map.items():
    orig_full = f"{orig_col}_original"
    api_full = f"{api_col}_api"
    qa_col = orig_col.split('.')[-1] + '_QA'

    # Only update if QA failed (i.e., 'No')
    validated_df[orig_col] = validated_df.apply(
        lambda row: row[api_col] if row.get(qa_col) in ["Not Matched","Missing in original source"] and pd.notna(row[api_col]) else row[orig_col],
        axis=1
    )

# Drop columns that end with _api or _QA
enriched_df = validated_df.drop(columns=[
    col for col in validated_df.columns if col.endswith('_api') or col.endswith('_QA')
])


##### - Add Company Age Column

In [ ]:
### incorporation age 
enriched_df['IncorporationDate'] = pd.to_datetime(enriched_df['IncorporationDate'], errors='coerce')
enriched_df['CompanyAgeYears'] = ((pd.Timestamp.today() - enriched_df['IncorporationDate']).dt.days // 365)

##### - Flag Overdue Accounts

In [ ]:
# Flag whether accounts or returns are overdue using NextDueDate
today = pd.Timestamp.today()
enriched_df['AccountsOverdue'] = pd.to_datetime(enriched_df['Accounts.NextDueDate'], errors='coerce') < today
enriched_df['ReturnsOverdue'] = pd.to_datetime(enriched_df['ReturnsNextDueDate'], errors='coerce') < today


##### - Add full Address column

In [ ]:
## Add full address
enriched_df['FullAddress'] = enriched_df['RegAddress.AddressLine1'].fillna('') + ", " + \
                    enriched_df['RegAddress.AddressLine2'].fillna('') + ", " + \
                    enriched_df['RegAddress.PostTown'].fillna('') + ", " + \
                    enriched_df['RegAddress.PostCode'].fillna('')

##### - Extract Code and Text

In [ ]:
# Extract code and text from 'SICCode.SicText_1'
df[['SICCode_1', 'SICText_1']] = df['SICCode.SicText_1'].str.extract(r'^(\d{4,})\s*-\s*(.+)$')
df[['SICCode_2', 'SICText_2']] = df['SICCode.SicText_2'].str.extract(r'^(\d{4,})\s*-\s*(.+)$')

# Optional: Handle 'NONE SUPPLIED' and other non-standard entries
df['SICCode_1'] = df['SICCode_1'].fillna('None Supplied')
df['SICText_1'] = df['SICText_1'].fillna('None Supplied')
df['SICCode_2'] = df['SICCode_2'].fillna('None Supplied')
df['SICText_2'] = df['SICText_2'].fillna('None Supplied')

### 8 - Reporting and visualisation

In [ ]:
# Show how many records passed vs. failed for each field.

In [ ]:
import matplotlib.pyplot as plt

qa_columns = [col for col in validated_df.columns if col.endswith('_QA')]

qa_summary = validated_df[qa_columns].apply(lambda col: col.value_counts().get('Not Matched', 0))
qa_summary.plot(kind='barh', color='salmon', figsize=(8, 6), title='Mismatches per Field')

plt.xlabel("Count of Mismatches")
plt.ylabel("Field")
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()


In [ ]:
# Count of companies with overdue filings

In [ ]:
enriched_df['AccountsOverdue'] = pd.to_datetime(enriched_df['Accounts.NextDueDate'], errors='coerce') < pd.Timestamp.today()
enriched_df['ReturnsOverdue'] = pd.to_datetime(enriched_df['ReturnsNextDueDate'], errors='coerce') < pd.Timestamp.today()

overdue_counts = enriched_df[['AccountsOverdue', 'ReturnsOverdue']].sum()
overdue_counts

In [ ]:
# Plot counts by PostTown.
enriched_df['RegAddress.PostTown'].value_counts().head(10).plot(kind='barh', color='steelblue')


In [ ]:
# Plot counts by PostTown.
enriched_df['CompanyStatus'].value_counts().head(10).plot(kind='barh', color='steelblue')


In [ ]:
# Plot counts by PostTown.
enriched_df['CompanyAgeYears'].value_counts().head(10).plot(kind='barh', color='steelblue')
